In [14]:

import json
import time
from web3 import Web3
from decimal import Decimal

w3 = Web3(Web3.HTTPProvider(f'https://api.securerpc.com/v1'))

def build_transaction_payload(
    action,
    web3,
    abi_path,
    contract_address,
    from_address,
    private_key,
    params
):
    """
    Builds transaction payloads for different actions.
    :param action: The action to perform ('swap', 'bridge', 'lend')
    :param web3: Web3 instance connected to a network
    :param abi_path: Path to the ABI file
    :param contract_address: Address of the contract
    :param from_address: Your wallet address
    :param private_key: Private key for signing transactions
    :param params: Dictionary of parameters required for the action
    :return: Raw transaction data ready to be signed and sent
    """

    
    with open(abi_path, 'r') as abi_file:
        contract_abi = json.load(abi_file)

    
    contract = web3.eth.contract(address=contract_address, abi=contract_abi)

    
    nonce = web3.eth.get_transaction_count(from_address)
    gas_price = web3.eth.gas_price  
    chain_id = web3.eth.chain_id    

    if action == 'swap':
        
        token_in = params['token_in']
        token_out = params['token_out']
        amount_in = params['amount_in']
        amount_out_min = params['amount_out_min']
        to_address = params['to_address']
        deadline = params.get('deadline', int(time.time()) + 600)  
        txn = contract.functions.swapExactTokensForTokens(
            amount_in,
            amount_out_min,
            [token_in, token_out],
            to_address,
            deadline
        ).build_transaction({
            'from': from_address,
            'nonce': nonce,
            'gasPrice': gas_price,
            'gas': 250000,  
            'chainId': chain_id,
        })

    elif action == 'lend':
        raise NotImplementedError("Lend action is not implemented yet.")
    elif action == 'bridge':
        raise NotImplementedError("Bridge action is not implemented yet.")
    else:
        raise ValueError(f"Unsupported action: {action}")
    signed_txn = web3.eth.account.sign_transaction(txn, private_key=private_key)
    return signed_txn.raw_transaction.hex() #return txn


def test_swap_on_sepolia():
    
    web3 = Web3(Web3.HTTPProvider('https://1rpc.io/sepolia'))

    
    from_address = web3.to_checksum_address('0x10EdEdD9Fa10D711F56d8451e8336aA2Bcd95517')
    private_key = ''     

    
    params = {
        'token_in': web3.to_checksum_address('0xFFf9976782D46Cc05630d1F6eBaB18B2324D6b14'),      
        'token_out': web3.to_checksum_address('0x7Fc21CEB0C5003576Ab5E101EB240C2B822C95D2'),    
        'amount_in': web3.to_wei(0.01, 'ether'),  
        'amount_out_min': 10,                 
        'to_address': from_address,          
        
    }

    
    abi_path = 'abis/IUniswapV2Router02.json'  
    contract_address = web3.to_checksum_address('0xeE567Fe1712Faf6149d80dA1E6934E354124CfE3')

    
    raw_txn = build_transaction_payload(
        action='swap',
        web3=web3,
        abi_path=abi_path,
        contract_address=contract_address,
        from_address=from_address,
        private_key=private_key,
        params=params
    )

    
    tx_hash = web3.eth.send_raw_transaction(raw_txn)
    print(f"Transaction sent with hash: {web3.to_hex(tx_hash)}")

    
    tx_receipt = web3.eth.wait_for_transaction_receipt(tx_hash)
    print(f"Transaction receipt: {tx_receipt}")

if __name__ == '__main__':
    test_swap_on_sepolia()


Transaction sent with hash: 0x8e8f9e38b9c7a50d139c299ac88a43878ef005dfcf412a442990002a72006e03
Transaction receipt: AttributeDict({'blockHash': HexBytes('0x568720de3c2fda87f865a59b6d9c2a262405f493b7f036a15e93d68b20644d95'), 'blockNumber': 7091613, 'contractAddress': None, 'cumulativeGasUsed': 7665404, 'effectiveGasPrice': 2179248, 'from': '0x10EdEdD9Fa10D711F56d8451e8336aA2Bcd95517', 'gasUsed': 27840, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 0, 'to': '0xeE567Fe1712Faf6149d8